<a href="https://colab.research.google.com/github/asmabahrini453/HexSoftwares_AI_Chatbot_Development/blob/main/FIREBOT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#libraries
import numpy as np
import nltk #for handeling  NLP tasks
import string #for string manipulation
import random

from sklearn.feature_extraction.text import TfidfVectorizer #convert text to numerical vectors  based on the Term Frequency-Inverse Document Frequency (TF-IDF) method.
from sklearn.metrics.pairwise import cosine_similarity #Measures the similarity between two vectors.

In [ ]:
#Read the text
txt = open('/content/sample_data/fire.txt','r',errors= 'ignore')
raw_doc = txt.read()

In [ ]:
raw_doc

'Fire in Your Workspace\n\nIf you have a safe evacuation route (minimal or no fire and/or smoke) from the area:\n\n- Stay calm and breathe deeply. Remember that panic can make it harder to think clearly.\n- Evacuate the building: to the designated emergency assembly area at a steady pace; there is no need to rush.\n- Pull the fire alarm and warn others nearby as you are leaving. Your alertness can help save others.\n- Close doors and windows if time permits; this can help slow down the spread of fire and smoke.\n- Move away from the fire and smoke whenever possible. Use the stairs, not the elevators.\n- Feel the doors with the back of your hand** to check for heat; **do not open them if they are hot.\n- Do not re-enter the building** until instructed to do so by emergency responders. Trust their expertise to guide you safely.\n- Once outside, from a safe location, call the campus Police Department at 197\n\nif your evacuation route has excessive smoke and/or heat, or the exit door is w

In [ ]:
#Pre-processing data

raw_doc= raw_doc.lower() #lower case conversion
nltk.download('punkt') #Tokenization
nltk.download('wordnet') #Downloads the WordNet database(only conatins english words)
nltk.download('omw-1.4') #Downloads the Open Multilingual WordNet (OMW)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
cleaned_doc = raw_doc.replace('\n\n', ' ').strip()

cleaned_doc

'fire in your workspace if you have a safe evacuation route (minimal or no fire and/or smoke) from the area: - stay calm and breathe deeply. remember that panic can make it harder to think clearly.\n- evacuate the building: to the designated emergency assembly area at a steady pace; there is no need to rush.\n- pull the fire alarm and warn others nearby as you are leaving. your alertness can help save others.\n- close doors and windows if time permits; this can help slow down the spread of fire and smoke.\n- move away from the fire and smoke whenever possible. use the stairs, not the elevators.\n- feel the doors with the back of your hand** to check for heat; **do not open them if they are hot.\n- do not re-enter the building** until instructed to do so by emergency responders. trust their expertise to guide you safely.\n- once outside, from a safe location, call the campus police department at 197 if your evacuation route has excessive smoke and/or heat, or the exit door is warm or ho

In [ ]:
sentence_tokens = nltk.sent_tokenize(cleaned_doc)
word_tokens = nltk.word_tokenize(cleaned_doc)

# Step 3: View the results
print(sentence_tokens[:2])  # First two sentences
print(word_tokens[0])

['fire in your workspace if you have a safe evacuation route (minimal or no fire and/or smoke) from the area: - stay calm and breathe deeply.', 'remember that panic can make it harder to think clearly.']
fire


In [ ]:
#lemmatization
lemmer = nltk.stem.WordNetLemmatizer() #lemmatization using the WordNet database

def LemTokens(tokens):
  return[lemmer.lemmatize(token) for token in tokens]
#removing  punctuation
#Creates a translation dictionary that maps each punctuation character to None
#effectively removing it using translate()
remove_punc_dict = dict((ord(punct), None) for punct in string.punctuation)

def LemNormalize(text):
  return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punc_dict)))

Define Greeting functions

In [ ]:
greet_inputs = ('hello', 'hi', 'help', 'emergency', 'fire', 'assist', 'urgent')
greet_responses = (
    "Emergency services activated. How can I assist you?",
    "I'm here to help. What is the nature of your emergency?",
    "Hello, please provide details of your emergency.",
    "This is the Fire Emergency Bot. How can I assist you?",
    "Stay calm. I'm here to help. What's the situation?"
)

def greet(sentence):
    for word in sentence.split():
        if word.lower() in greet_inputs:
            return random.choice(greet_responses)
    return None

In [ ]:
# Function to handle evacuation and other emergency instructions
def emergency_response(user_input):
    instructions = {
        "evacuate": "Evacuate the building: to the designated emergency assembly area at a steady pace; there is no need to rush.",
        "fire": "If there's a fire, leave the building immediately and call emergency services.",
        "help": "If you need help, contact emergency services right away and provide your location."
        # Add more instructions as needed
    }

    for keyword in instructions:
        if keyword in user_input:
            return instructions[keyword]
    return None

Response Generation

In [ ]:

def response(user_input):
    bot_response = ''
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
    tfidf = TfidfVec.fit_transform(sentence_tokens)

    user_input_vec = TfidfVec.transform([user_input])
    vals = cosine_similarity(user_input_vec, tfidf)

    idx = vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]

    if req_tfidf == 0:
        return "I am sorry. Unable to understand you!"
    else:
        return sentence_tokens[idx]

Defining the chatFlow

In [ ]:
flag = True
print('Hello! I am FIREBOT.')
while flag:
    user_input = input().lower()
    if user_input != 'bye':
        if user_input in ['thank you', 'thanks']:
            flag = False
            print('FIREBOT: You are welcome.')
        else:
            greet_response = greet(user_input)
            if greet_response:
                print('FIREBOT: ' + greet_response)
            else:
                emergency_instruction = emergency_response(user_input)
                if emergency_instruction:
                    print('FIREBOT: ' + emergency_instruction)
                else:
                    sentence_tokens.append(user_input)
                    word_tokens += nltk.word_tokenize(user_input)
                    final_words = list(set(word_tokens))
                    print('FIREBOT: ' + response(user_input))
                    sentence_tokens.remove(user_input)
    else:
        flag = False
        print('FIREBOT: Goodbye!')

Hello! I am FIREBOT.
FIREBOT: This is the Fire Emergency Bot. How can I assist you?
FIREBOT: Evacuate the building: to the designated emergency assembly area at a steady pace; there is no need to rush.
FIREBOT: Hello, please provide details of your emergency.


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn(


FIREBOT: fire in your workspace if you have a safe evacuation route (minimal or no fire and/or smoke) from the area: - stay calm and breathe deeply.
FIREBOT: Stay calm. I'm here to help. What's the situation?
FIREBOT: Emergency services activated. How can I assist you?
FIREBOT: by following these steps, you can remain calm and safe during a fire emergency, knowing that help is on the way and there are clear actions to protect yourself and others.
FIREBOT: I am sorry. Unable to understand you!
